In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import math
from numba import njit,jit
import time
from matplotlib import pyplot as plt
import yaml
import networkx as nx
import scipy
import multiprocessing
from joblib import Parallel, delayed

In [4]:
plt.rc('font', family='sans-serif')
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')
plt.close('all')
cols = ['c','m','b','g','r','y','k']

# Define paths
savepath = Path("/Users/saadjansari/Desktop/Meetings/Flatiron/Flatiron 201112")
path_prefix = Path("/Users/saadjansari/Documents/Projects/Results/AMSOS")
simpaths = [
    path_prefix / "Tactoids/scan_filamin_6400/run/f5_merged",
    path_prefix / "Tactoids/scan_filamin_6400/run/f10_merged",
]
# simpaths = [
#     path_prefix / "Tactoids/n25600/f5_merged",
# ]
cols = cols[0:len(simpaths)]
# resolve paths
for spath in simpaths:
    spath.resolve()

In [5]:
@njit
def unfold_trajectories(crds, boxsize):
    # unfolded trajectory via the shortest route
    # expect input: crds = [time, particle, dim]
    crds2 = np.zeros_like(crds)
    crds2[0,:,:] = crds[0,:,:]
    for idx in np.arange(crds.shape[-1]):
        for ipart in np.arange(crds.shape[1]):
            
            # find amount particle travels frame to frame
            dist = nb_diff( crds[:,ipart,idx])
            k = np.sign( dist) * np.floor( np.absolute(dist)/(0.51*boxsize[idx]))
            dist -= k*boxsize[idx]
            crds2[1:,ipart,idx] = dist.cumsum()+crds[0,ipart,idx]
    return crds2

@njit
def nb_diff(x):
    return x[1:] - x[:-1]

def get_boxsize_pbc(spath, conf=False):
    # Read yaml config file to extract information about space
    # If confinement is True, use a cylindrical confinement in XY plane, but full box size in Z
    with open( spath / 'RunConfig.yaml') as yf:
        rconfig = yaml.safe_load( yf)
        
    box_size = np.array( rconfig['simBoxHigh']) - np.array( rconfig['simBoxLow'])
    if conf:
        box_size[0] = 2*box_size[0]
        box_size[1] = 2*box_size[1]
    return box_size

def find_frames(spath):
    
    def fileKey(f):
        return int( f.parts[-1].split('_')[-1].split('.dat')[0] )
    file_s_last = sorted( list(spath.glob('**/SylinderAscii*')), key=fileKey)
    file_p_last = sorted( list(spath.glob('**/ProteinAscii*')), key=fileKey)
    return file_s_last, file_p_last

def read_dat_sylinder_centers( fname):
    # Read a SylinderAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Initialize numpy arrays for data
        pos0 = np.zeros( (len(filecontent)-2,3))
        pos1 = np.zeros( (len(filecontent)-2,3))
        c = np.zeros( (len(filecontent)-2,3))

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []
        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()

            dat = np.array( list(map(float,data[2::])) )
            c[idx,:] = (dat[1:4] + dat[4::])/2            

    return c

def get_nodes_in_clusters( nodes, edge0, edge1, min_size_ratio=0.1):
    # Get largesst connected component
    # nodes is a list of integers representing node indices
    # edge0 / edge1 are the list of nodes that each edge connects to.
    # Indices whose values are -1 are ignored in both edge0 and edge1.

    # Create a graph for filaments
    g = nx.Graph()
    g.add_nodes_from( nodes)

    # add edges to the graph
    for e0,e1 in zip( edge0,edge1):
        if e0 != -1 and e1 != -1:
            g.add_edge(e0, e1)

    # find connected component largest
    cc_sort = sorted(nx.connected_components(g), key=len, reverse=True)
    cc_large = [ii for ii in cc_sort if len(ii) > min_size_ratio*len(nodes)]
    cc = []
    for clus in cc_large:
        cc+=clus

    # also get a boolean array representing nodes that are in the largest cc
    cc_bool = np.zeros(len(nodes), dtype=bool)
    cc_bool[ cc] = True
    return cc, cc_bool

def read_dat_sylinder( fname):
    # Read a SylinderAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Initialize numpy arrays for data
        gids = np.zeros( len(filecontent)-2, dtype=int)
        rad = np.zeros(len(filecontent)-2)
        pos0 = np.zeros( (len(filecontent)-2,3))
        pos1 = np.zeros( (len(filecontent)-2,3))
        ort = np.zeros( (len(filecontent)-2,3))

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []
        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()
            gids[idx] = int(data[1])

            dat = np.array( list(map(float,data[2::])) )
            rad[idx] = dat[0]
            pos0[idx,:] = dat[1:4]
            pos1[idx,:] = dat[4::]
            xi = pos1[idx,:] - pos0[idx,:]
            ort[idx,:] =  xi/np.sqrt(xi.dot(xi))

    # Store data as a dataframe
    df = pd.DataFrame({
        'gid': gids,
        'radius' : rad,
        'pos0' : list(pos0),
        'pos1': list(pos1),
        'orientation': list(ort)
        })
    return df

def read_dat_protein( fname):
    # Read a ProetinAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []

        # Initialize numpy arrays for data
        gids = np.zeros( len(filecontent), dtype=int)
        pos0 = np.zeros( (len(filecontent),3))
        pos1 = np.zeros( (len(filecontent),3))
        link0 = np.zeros( len(filecontent), dtype=int)
        link1 = np.zeros( len(filecontent), dtype=int)

        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()
            # pdb.set_trace()
            gids[idx] = int(data[1])
            link0[idx] = int(data[9])
            link1[idx] = int(data[10])
            dat = np.array( list(map(float,data[2:9])) )
            pos0[idx,:] = dat[1:4]
            pos1[idx,:] = dat[4::]

    # Store data as a dataframe
    df = pd.DataFrame({
        'gid': gids,
        'pos0' : list(pos0),
        'pos1': list(pos1),
        'link0': link0,
        'link1': link1
        })
    return df

def load_trajectories(spath):

    # Load data
    boxsize = get_boxsize_pbc( spath)

    # Get files for frames
    files_syl, files_prot = find_frames( spath)
    nT = len(files_syl)
    nDim = 3

    # number of particles
    nPar = len( read_dat_sylinder_centers(files_syl[0]))

    # Load position centers and store them in a numpy array
    pos = np.zeros( (nT, nPar, nDim))
    for idx,fil in enumerate( files_syl):
        pos[idx,:,:] = read_dat_sylinder_centers(fil)

    # Load 10 frames for sylinder, protein to get cluster elements
    frr = np.array( np.floor(np.linspace(0,nT-1, 10)), dtype=int)
    cc_bool = np.ones((1,nPar),dtype=bool)
    for fr in frr:
        
        df_sylinder = read_dat_sylinder(files_syl[fr])
        df_protein = read_dat_protein( files_prot[fr])
        cc, cc_booll = get_nodes_in_clusters(
                df_sylinder.gid.tolist(),
                df_protein.link0.tolist(),
                df_protein.link1.tolist(),
                min_size_ratio=0.1)
        cc_bool = np.logical_and(cc_bool, cc_booll)
    
    cc = np.argwhere(cc_bool)[:,1].tolist()
    return pos[:,cc,:], boxsize

@njit
def msd_traj_1d(pos):
    result = np.zeros_like(pos)
    deltastop = pos.shape[0]
    for traj in range(pos.shape[1]):
#         for dim in range(pos.shape[2]):
        for delta in range(1,deltastop):
            thisresult = 0
            for i in range(delta,deltastop):
#                 thisresult += (pos[traj,i,dim] - pos[traj,i-delta,dim])**2
                thisresult += (pos[i,traj] - pos[i-delta,traj])**2
#             result[traj,delta,dim] = thisresult / (deltastop - delta)
            result[delta,traj] = thisresult / (deltastop - delta)
    return result

@njit
def msd_traj_3d(pos):
    msd = np.zeros_like(pos)
    for idx in np.arange(pos.shape[-1]):
        msd[:,:,idx] = msd_traj_1d(pos[:,:,idx])
    return msd

In [9]:
msds = []

# Analyze each sim
spath = simpaths[0]

t0 = time.time()
pos, boxsize = load_trajectories(spath)
print('Elapsed time = {}'.format(time.time()-t0))

# for spath in simpaths:

#     # Load trajectory
#     pos, boxsize = load_trajectories(spath)
    
#     # Unfold trajectory
#     pos_full = unfold_trajectories(pos, boxsize)
    
#     # Calculate MSD
#     msds.append( msd_traj_3d(pos_full) )

Elapsed time = 101.84080171585083


In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
for spath, msd, col in zip(simpaths, msds, cols):
    
    # Plot MSD_z
    ax.plot( np.mean(msd[:,:,2], axis=1), label='{0}_Z'.format(spath.name), ls='-', color=col)
    
    ax.plot( 
        np.mean( 0.5*(msd[:,:,0] + msd[:,:,1]), axis=1), 
        label='{0}_XY'.format(spath.name), ls='--', color=col)
    
plt.legend()
ax.set(xlabel=r'timelag ($\tau$)', ylabel=r'$MSD$ ($\mu m^2$)')
plt.savefig( savepath/"Tac_msd.pdf")

In [8]:
num_cores = multiprocessing.cpu_count()
num_cores

8

In [18]:
inputs = range(1000000) 
def processInput(i):
    return i * i

num_cores = multiprocessing.cpu_count()

%timeit Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
#print(results)

3.5 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit [processInput(i) for i in inputs]


179 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
import asyncio

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def read_dat_sylinder_centers( fname):
    # Read a SylinderAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Initialize numpy arrays for data
        pos0 = np.zeros( (len(filecontent)-2,3))
        pos1 = np.zeros( (len(filecontent)-2,3))
        c = np.zeros( (len(filecontent)-2,3))

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []
        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()

            dat = np.array( list(map(float,data[2::])) )
            c[idx,:] = (dat[1:4] + dat[4::])/2            

    return c

def load_trajectories(spath):

    # Load data
    boxsize = get_boxsize_pbc( spath)

    # Get files for frames
    files_syl, files_prot = find_frames( spath)
    nT = len(files_syl)
    nDim = 3

    # number of particles
    nPar = len( read_dat_sylinder_centers(files_syl[0]))

    # Load position centers and store them in a numpy array
    pos = np.zeros( (nT, nPar, nDim))
    for idx,fil in enumerate( files_syl):
        pos[idx,:,:] = read_dat_sylinder_centers(fil)

    # Load 10 frames for sylinder, protein to get cluster elements
    frr = np.array( np.floor(np.linspace(0,nT-1, 10)), dtype=int)
    cc_bool = np.ones((1,nPar),dtype=bool)
    for fr in frr:
        
        df_sylinder = read_dat_sylinder(files_syl[fr])
        df_protein = read_dat_protein( files_prot[fr])
        cc, cc_booll = get_nodes_in_clusters(
                df_sylinder.gid.tolist(),
                df_protein.link0.tolist(),
                df_protein.link1.tolist(),
                min_size_ratio=0.1)
        cc_bool = np.logical_and(cc_bool, cc_booll)
    
    cc = np.argwhere(cc_bool)[:,1].tolist()
    return pos[:,cc,:], boxsize

t0 = time.time()
pos, boxsize = load_trajectories(spath)
print('Elapsed time = {}'.format(time.time()-t0))

TypeError: object of type '_asyncio.Future' has no len()